# Credit Card Fraud Detection With Machine Learning in Python

This example shows use of classification to help credit card company to detect potential fraud cases using the HPC-like platform Bodo. Credit card data is extracted from Bodo's public S3 bucket, cleaned and processed. Then some analysis are done to extract insight. All are **parallelized across multiple cores using Bodo**. This can be a straightforward way to make Python code run faster without a lot of changes to the code. 
Original example can be found [here](https://medium.com/codex/credit-card-fraud-detection-with-machine-learning-in-python-ac7281991d87).

You can run the large-scale example on [Bodo platform](https://platform.bodo.ai/account/login).

The current results are based on running on 2 `c5.2xlarge` instances (8 cores, 32GiB memory)

The dataset was downloaded from Kaggle [here](https://www.kaggle.com/mlg-ulb/creditcardfraud) and save in parquet format in S3 Bucket (`s3://bodo-example-data/creditcard/creditcard.pq/`)
The example using CSV dataset can be found in [Bodo-Examples Git repository](https://github.com/Bodo-inc/Bodo-examples/blob/master/ml/credit-card-fraud.ipynb)

The Bodo framework knows when to parallelize code based on the `%%px` at the start of cells and `@bodo.jit` function decorators. Removing those and restarting the kernel will run the code without Bodo.

## Importing the Packages

These are the main packages we are going to work with:
 - Bodo to parallelize Python code automatically
 - Pandas to work with data
 - Numpy to work with arrays
 - scikit-learn to build and evaluate classification models
 - xgboost for xgboost classifier model algorithm

In [1]:
%%px
import warnings
warnings.filterwarnings("ignore")

import bodo
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler # data normalization
from sklearn.model_selection import train_test_split # data split
from sklearn.linear_model import LogisticRegression # Logistic regression algorithm
from sklearn.ensemble import RandomForestClassifier # Random forest tree algorithm
from xgboost import XGBClassifier # XGBoost algorithm
from sklearn.svm import LinearSVC # SVM classification algorithm
from sklearn.metrics import accuracy_score # evaluation metric
print(f"Hello World from rank {bodo.get_rank()}. Total ranks={bodo.get_size()}")

Starting 8 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:1] Hello World from rank 1. Total ranks=8


[stdout:3] Hello World from rank 3. Total ranks=8


...

[stdout:2] Hello World from rank 2. Total ranks=8


[stdout:4] Hello World from rank 4. Total ranks=8


[stdout:5] Hello World from rank 5. Total ranks=8


[stdout:6] Hello World from rank 6. Total ranks=8


[stdout:0] Hello World from rank 0. Total ranks=8


## Data Processing and EDA
1. Load dataset
2. Compute the percentage of fraud cases in the overall recorded transcations.
3. Get a statistical view of both fraud and non-fraud transaction amount data

In [2]:
%%px
@bodo.jit(distributed=['df'], cache=True)
def load_data():
    start = time.time()
    df = pd.read_parquet('s3://bodo-example-data/creditcard/creditcard.pq')
    df.drop('Time', axis = 1, inplace = True)
    end = time.time()
    print("Read Time: ", (end-start))
    print(df.shape)
    return df

df = load_data()

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] Read Time:  2.1310838393079834
(284807, 30)


In [3]:
%%px
@bodo.jit(distributed=['df'], cache=True)
def data_processing(df):
    cases = len(df)
    nonfraud_cases = df[df.Class == 0]
    fraud_cases = df[df.Class == 1]
    nonfraud_count = len(nonfraud_cases)
    fraud_count = len(fraud_cases)
    fraud_percentage = round(fraud_count / nonfraud_count * 100, 2)
    print("--------------------------------------------")
    print("Total number of cases are ", cases)
    print("Number of Non-fraud cases are ", nonfraud_count)
    print("Number of fraud cases are", fraud_count)
    print("Percentage of fraud cases is ", fraud_percentage)
    print("--------------------------------------------")
    print("--------------------------------------------")
    print("NON-FRAUD CASE AMOUNT STATS")
    print(nonfraud_cases.Amount.describe())
    print("FRAUD CASE AMOUNT STATS")
    print(fraud_cases.Amount.describe())
    print("--------------------------------------------")  

data_processing(df)

[stdout:0] --------------------------------------------
Total number of cases are  284807
Number of Non-fraud cases are  284315
Number of fraud cases are 492
Percentage of fraud cases is  0.17
--------------------------------------------
--------------------------------------------
NON-FRAUD CASE AMOUNT STATS
count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64
FRAUD CASE AMOUNT STATS
count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64
--------------------------------------------


## Feature Selection & Data Split

### 1. Normalize `Amount` variable
`Amount` variable varies when compared to the rest of the variables. To reduce its range of values, we normalize it using the `StandardScaler` 

In [4]:
%%px
@bodo.jit(distributed=['df'], cache=True)
def sc(df):
    start = time.time()    
    sc = StandardScaler()
    amount = df['Amount'].values
    amount = amount.reshape(-1,1)
    sc.fit(amount)
    df['Amount'] = (sc.transform(amount)).ravel()
    print("StandardScaler time: ", time.time() - start)
    return df['Amount'].head(10)

sc_df = sc(df)
if bodo.get_rank() == 0:
    display(sc_df)

[stdout:0] StandardScaler time:  0.13710344474846892


[output:0]

0    0.244964
1   -0.342475
2    1.160686
3    0.140534
4   -0.073403
5   -0.338556
6   -0.333279
7   -0.190107
8    0.019392
9   -0.338516
Name: Amount, dtype: float64

### 2. Split the data into a training set and testing set 

In [5]:
%%px
@bodo.jit(distributed=['df', 'X_train', 'X_test', 'y_train', 'y_test'], cache=True)
def data_split(df):
    X = df.drop('Class', axis = 1).values
    y = df['Class'].values.astype(np.int64)
    start = time.time()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, train_size=0.8, random_state = 0)
    print("train_test_split time: ", time.time() - start)    
    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = data_split(df)
if bodo.get_rank() == 0:
    print('X_train samples :', X_train[:1])
    print('X_test samples :', X_test[0:1])
    print('y_train samples :', y_train[0:20])
    print('y_test samples :', y_test[0:20])    

[stdout:0] train_test_split time:  0.06754623915503544
X_train samples : [[ 1.26585198 -0.1075269   0.47981009 -0.25184768 -0.60788305 -0.52676387
  -0.34531619  0.02365571  0.21786475 -0.03118412  1.10872232  0.59520842
  -0.28786759  0.38413387  0.61749279  0.71709663 -0.71275056  0.23189477
   0.44609545 -0.07168675 -0.10932857 -0.35212209  0.04888506  0.03685774
   0.11879204  0.90961715 -0.07721121 -0.00528953 -0.34963112]]
X_test samples : [[-0.2500975   0.86416881  1.71780059  0.42077176  0.45859239 -0.57885514
   0.94800926 -0.51391496 -0.31043625 -0.02031068  0.55475393  0.09970827
   0.43210406 -0.79585124  1.45354931 -0.47625902  0.39926659 -0.69608176
   0.13674418  0.14104106 -0.19687718 -0.20118382  0.02023838  0.37742844
  -0.90739478  0.02715091 -0.29036732 -0.27941574 -0.33619755]]
y_train samples : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_test samples : [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Modeling
Here we have built four different types of classification models and evaluate these models using accuracy score metrics provided by scikit-learn package.

#### 1. Logistic Regression

In [6]:
%%px
@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'], cache=True)
def lr_model(X_train, X_test, y_train, y_test):
    start = time.time()
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    lr_yhat = lr.predict(X_test)
    print("LogisticRegression fit and predict time: ", time.time()-start)
    print('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat)))
    
    
lr_model(X_train, X_test, y_train, y_test)

[stdout:0] LogisticRegression fit and predict time:  0.6353843132923984
Accuracy score of the Logistic Regression model is 0.9987008883115059


#### 2. Random Forest Tree

In [7]:
%%px
@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'], cache=True)
def rf_model(X_train, X_test, y_train, y_test):
    start = time.time()
    rf = RandomForestClassifier(max_depth = 4)
    rf.fit(X_train, y_train)
    rf_yhat = rf.predict(X_test)
    print("RandomForestClassifier fit and predict time: ", time.time()-start)    
    print('Accuracy score of the Random Forest Tree model is {}'.format(accuracy_score(y_test, rf_yhat)))

rf_model(X_train, X_test, y_train, y_test)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] RandomForestClassifier fit and predict time:  17.153762485349738
Accuracy score of the Random Forest Tree model is 0.9988237772550121


#### 3. XGBoost Model

In [8]:
%%px
@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'], cache=True)
def xgb_model(X_train, X_test, y_train, y_test):  
    start = time.time()
    xgb = XGBClassifier(max_depth = 4)
    xgb.fit(X_train, y_train)
    xgb_yhat = xgb.predict(X_test)
    print("XGBClassifier fit and predict time: ", time.time()-start) 
    print('Accuracy score of the XGBoost model is {}'.format(accuracy_score(y_test, xgb_yhat)))

xgb_model(X_train, X_test, y_train, y_test)

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier fit and predict time:  3.700095154756724
Accuracy score of the XGBoost model is 0.9989115550718023


[stdout:2] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[stdout:4] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[stdout:6] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[stdout:1] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[stdout:5] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[stdout:3] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[stdout:7] [23:06:14] WARNING: /xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


#### 4. SVM

In [9]:
%%px
@bodo.jit(distributed=['X_train', 'y_train', 'X_test', 'y_test'], cache=True)
def lsvc_model(X_train, X_test, y_train, y_test):  
    start = time.time()
    lsvc = LinearSVC(random_state=42)
    lsvc.fit(X_train, y_train)
    lsvc_yhat = lsvc.predict(X_test)
    print("LinearSVC fit and predict time: ", time.time()-start) 
    print('Accuracy score of the Linear Support Vector Classification model is {}'.format(accuracy_score(y_test, lsvc_yhat)))

lsvc_model(X_train, X_test, y_train, y_test)

[stdout:0] LinearSVC fit and predict time:  0.4209893488538796
Accuracy score of the Linear Support Vector Classification model is 0.9987008883115059
